# Import Libraries

In [1]:
from swarmintelligence import WhaleOptimizationAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/WOA/optimal_params.pkl', 'rb'))
optimal_params

{'n_whales': 25, 'maxIteration': 100, 'constanta': 5}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
woa_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                woa_evaluation_results['image_name'].append(image_name)
                woa_evaluation_results['thresholds'].append(threshold)
                woa_evaluation_results['fitness_function'].append(fitness_function)
                woa_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # WOA optimization
                    woa = WhaleOptimizationAlgorithm(
                        k=threshold,
                        n_whales=optimal_params['n_whales'],
                        maxIteration=optimal_params['maxIteration'],
                        constanta=optimal_params['constanta'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    whales, best_thresholds = woa.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(woa.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(woa.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                woa_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                woa_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                woa_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                woa_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                woa_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                woa_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                woa_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                woa_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                woa_evaluation_results['Regions'].append(mean_regions)
                woa_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                woa_evaluation_results['Fitness'].append(fitness_values)
                woa_evaluation_results['MSE'].append(list_mse)
                woa_evaluation_results['RMSE'].append(list_rmse)
                woa_evaluation_results['PSNR'].append(list_psnr)
                woa_evaluation_results['SSIM'].append(list_ssim)
                woa_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
12.211418116190842
12.211418116190842
12.211474090566005
12.211120689196022
12.211474090566005
12.211418116190842
12.211474090566005
12.21057914485725
12.211474090566005
12.211418116190842
12.211474090566005
12.211474090566005
12.210612503695163
12.211120689196022
12.211474090566005
12.210119740150311
12.211474090566005
12.211474090566005
12.211474090566005
12.200771098533782
12.211474090566005
12.211131163036912
12.210612503695163
12.211474090566005
12.211418116190842
12.21057914485725
12.205762282971317
12.211474090566005
12.211474090566005
12.211474090566005
Mean Fitness = 12.210720363321117
	 AirplaneF16.tiff | Threshold =  3
15.499240040923393
15.500850438618196
15.503581777339408
15.503893789532656
15.499798305342695
15.498941061991886
15.501166321222817
15.495203745692594
15.500850438618196
15.49952941270475
15.503893789532656
15.503893789532656
15.503581777339408
15.503210052181657
15.499547003986738
15.503581777339408
15.503210052181657
15.4

21.329576179779757
21.57000696975611
21.49282337192884
21.546808721627862
Mean Fitness = 21.46913249693986
	 Mandrill.tiff | Threshold =  2
12.216108690500446
12.217727299731392
12.217754102632629
12.217754102632629
12.217754102632629
12.217727299731392
12.217754102632629
12.217500675014861
12.217727299731392
12.217727299731392
12.217727299731392
12.217754102632629
12.217754102632629
12.217754102632629
12.217727299731392
12.217754102632629
12.217754102632629
12.217754102632629
12.217727299731392
12.217727299731392
12.217500675014861
12.217500675014861
12.216715665461326
12.217754102632629
12.216386093476034
12.217500675014861
12.217727299731392
12.217727299731392
12.217754102632629
12.217754102632629
Mean Fitness = 12.217576316034508
	 Mandrill.tiff | Threshold =  3
15.279196147154849
15.278667738059248
15.276863297432058
15.278399000437851
15.278667738059248
15.279196147154849
15.27414087802099
15.279177754923928
15.277998159751782
15.278909017302531
15.279177754923928
15.278909017302

20.973712539843188
21.006072094900023
20.909219572539886
20.974698131753534
20.886503034427026
20.974201261266074
20.925232711958355
21.01737055285302
20.921249958967167
20.95249692329456
20.993201121617428
20.8897469213913
21.004919861646158
Mean Fitness = 20.939528381815684


## Visualize Results using DataFrame

In [7]:
woa_evaluation_results_df =pd.DataFrame(woa_evaluation_results)
woa_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[71, 173]",4.393207,12.210720,2797.375213,52.889014,13.663696,0.761930,0.922145,"[[71, 255, 255, 255, 255, 255, 255, 255, 255, ...","[3.854238986968994, 3.216564893722534, 4.25249...","[12.211418116190842, 12.211418116190842, 12.21...","[2797.494888305664, 2797.494888305664, 2806.37...","[52.89134984386071, 52.89134984386071, 52.9752...","[13.66311059227012, 13.66311059227012, 13.6493...","[0.7620556133220606, 0.7620556133220606, 0.761...","[0.9221686614104341, 0.9221686614104341, 0.921..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[69, 126, 182]",4.085422,15.501507,2005.198348,44.779040,15.109386,0.798957,0.949370,"[[69, 255, 255, 255, 182, 255, 255, 255, 255, ...","[2.8677570819854736, 4.1704421043396, 5.045282...","[15.499240040923393, 15.500850438618196, 15.50...","[2015.1718292236328, 2023.49462890625, 1995.86...","[44.89066527936106, 44.98327054479532, 44.6750...","[15.087682774865483, 15.069783049149594, 15.12...","[0.7996103860873256, 0.8015029856770503, 0.797...","[0.9491893767171636, 0.9492128600303328, 0.949..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[63, 102, 142, 185]",3.939853,18.297587,1839.450730,42.888252,15.484151,0.813492,0.956656,"[[102, 255, 255, 255, 185, 255, 255, 255, 255,...","[4.852482318878174, 4.58064079284668, 5.368779...","[18.30638170762925, 18.304858805930927, 18.272...","[1852.5834732055664, 1852.6384086608887, 1877....","[43.04164812371346, 43.042286285243826, 43.332...","[15.453025754522603, 15.452896973219515, 15.39...","[0.8150293650486193, 0.8150936680676173, 0.815...","[0.9563852983298609, 0.956422624325286, 0.9559..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[55, 86, 120, 153, 187]",4.548793,20.866783,1733.264298,41.630719,15.742895,0.814384,0.961041,"[[86, 255, 255, 187, 187, 187, 255, 255, 255, ...","[5.16283655166626, 3.1257426738739014, 3.07946...","[20.8893243769476, 20.90472603732294, 20.86304...","[1672.2789878845215, 1748.6170043945312, 1756....","[40.89350789409636, 41.816468100433006, 41.910...","[15.89771627961478, 15.703856635061433, 15.684...","[0.8076403244804229, 0.8178694537148833, 0.816...","[0.9619559827264212, 0.9604119495051165, 0.960..."
4,Lena.png,2,kapur_entropy,max,"[97, 164]",4.114759,12.345994,2045.126571,45.222968,15.023643,0.631273,0.891644,"[[164, 164, 164, 164, 164, 164, 164, 164, 255,...","[3.025892734527588, 5.799316883087158, 3.51887...","[12.346365101175513, 12.346365101175513, 12.34...","[2044.6897659301758, 2044.6897659301758, 2030....","[45.21824594044063, 45.21824594044063, 45.0661...","[15.02452937602985, 15.02452937602985, 15.0537...","[0.6307635952929036, 0.6307635952929036, 0.633...","[0.8916733985521984, 0.8916733985521984, 0.891..."
5,Lena.png,3,kapur_entropy,max,"[81, 126, 175]",4.039004,15.316759,1211.190179,34.800587,17.299461,0.706308,0.933287,"[[175, 175, 175, 175, 175, 175, 175, 175, 175,...","[4.717473030090332, 4.001605272293091, 6.19329...","[15.317273951521774, 15.31822734967293, 15.318...","[1197.7980461120605, 1201.5421257019043, 1205....","[34.60921909133548, 34.66326767201708, 34.7271...","[17.3469676055472, 17.333413592284728, 17.3174...","[0.706253230183056, 0.705610164794634, 0.70490...","[0.9342425464635277, 0.9335011139255196, 0.932..."
6,Lena.png,4,kapur_entropy,max,"[65, 98, 138, 179]",4.423885,18.006711,862.883935,29.370202,18.774093,0.745891,0.958353,"[[179, 179, 179, 179, 179, 179, 179, 179, 179,...","[3.923393964767456, 3.270298480987549, 3.24031...","[18.01102696560244, 18.00599396198431, 18.0065...","[868.578727722168, 883.7286834716797, 839.0249...","[29.471659738164867, 29.727574463310653, 28.96...","[18.742711719757178, 18.667614096072537, 18.89...","[0.7456528088764203, 0.7470067897766614, 0.742...","[0.9589194367867114, 0.9583704781147732, 0.958..."
7,Lena.png,5,kapur_ent

# Save results

In [8]:
pickle.dump(woa_evaluation_results_df, open('results/evaluation/WOA/WOA_kapur_df.pkl', 'wb'))
pickle.dump(woa_evaluation_results, open('results/evaluation/WOA/WOA_kapur_dict.pkl', 'wb'))